In [1]:
import json
import requests
import pandas as pd

In [2]:
# get all the symbols and store in df
symbol_url = 'https://api.iextrading.com/1.0/ref-data/symbols/'
r = requests.get(symbol_url)
symbols_df = pd.read_json(json.dumps(r.json()))
data = pd.DataFrame()

set_index=True

for i in symbols_df.itertuples():
    symbol = i.symbol
    symbol_type = i.type
    if symbol_type == 'cs':
        stock_url = 'https://api.iextrading.com/1.0/stock/{}/chart/5y?filter=date,close'.format(symbol)
        stock_request = requests.get(stock_url)
        single_stock_df = pd.read_json(json.dumps(stock_request.json()))
        if 'date' in single_stock_df.columns:
            if set_index:
                data['date'] = single_stock_df['date']
                data[symbol] = single_stock_df['close']
                data.rename(index=str, columns={symbol: symbol}, inplace=True)
                data.set_index('date', inplace=True)
                set_index = False
            else:
                single_stock_df.rename(index=str, columns={'close': symbol}, inplace=True)
                single_stock_df.set_index('date', inplace=True)
                data = data.join(single_stock_df, on='date', how='left')
        else:
            print('Bad symbol: {}'.format(symbol))

data.to_csv('/Users/niejo05/learning-lppl/data/backtest.csv')


Bad symbol: BFL
Bad symbol: LTN^
